# Clustering

El clustering o agrupamiento es la tarea de agrupar una serie de objetos de una manera que objetos del mismo grupo (cluster) son más similares entre sí que con los de otros grupos. Su objetivo principal es en la fase exploratoria de los datos y se clasifica dentro de los métodos no supervisados de aprendizaje de máquinas.

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import sklearn.cluster # Algoritmos de clustering
import sklearn.metrics
import scipy.cluster
import seaborn as sns

In [ ]:
blobs = pd.read_csv("https://raw.githubusercontent.com/fvillena/biocompu/2022/data/blobs.csv") # Importamos un csv que contiene los datos a analizar

In [ ]:
blobs.head() # Verificamos qué podemos encontrar en el conjunto de datos.

In [ ]:
plt.scatter(blobs.x, blobs.y) # Exploramos la distribución que tienen nuestros datos

Intuitivamente podemos desprender que existe cierto agrupamiento de los datos. Uno de los algoritmos para realizar clustering es el k-means que se encuentra implementado en la clase sklearn.cluster.KMeans. Procederemos a realizar un agrupamiento utilizando este algoritmo.

k-Means necesita que nosotros asignemos el hiperparámetro de la cantidad de grupos a ajustar, comenzaremos con 2 grupos.

Ajustamos un modelo de k-means.

In [ ]:
km = sklearn.cluster.KMeans(n_clusters=2, random_state=11) # Instanciamos el algoritmo de k-means
km.fit(blobs) # Ajustamos el modelo de k-means

In [ ]:
km.labels_ # Grupos a los cuales asoció el modelo a cada una de las instancias del conjunto de datos

In [ ]:
plt.scatter(blobs.x, blobs.y, c = km.labels_) # Verificamos gráficamente el agrupamiento

Utilizaremos el método del codo para intentar encontrar la cantidad de grupos óptima en nuestro conjunto de datos.

El método del codo busca encontrar la cantidad óptima de grupos al iterar por un rango de cantidad de grupos para encontrar dónde se genera un cambio en la pendiente de la curva de dispersión de los grupos contra la cantidad de grupos.

In [ ]:
inertias = []
n_clusters_iterable = range(1,10)
for n_clusters in n_clusters_iterable:
    km = sklearn.cluster.KMeans(n_clusters=n_clusters) # Instanciamos el algoritmo de k-means
    km.fit(blobs) # Ajustamos el modelo de k-means
    inertias.append(km.inertia_) # Esta la dispersión de nuestros grupos

In [ ]:
plt.plot(
    n_clusters_iterable,
    inertias
)

Otro método de optimización de la cantidad de grupos es el método de la silueta, en donde se busca maximizar el silhouette score al modular el número de clusters.

El silhouette score es un promedio de las siluetas de cada punto. Cada silueta se calcula usando la distancia intra-cluster promedio $a$ y la distancia promedio al cluster más cercano $b$, por lo tanto el valor de la silueta es:

$\frac{b-a}{max(a,b)}$

In [ ]:
silhouette_scores = []
n_clusters_iterable = range(2,10)
for n_clusters in n_clusters_iterable:
    km = sklearn.cluster.KMeans(n_clusters=n_clusters) # Instanciamos el algoritmo de k-means
    km.fit(blobs) # Ajustamos el modelo de k-means
    silhouette_scores.append(sklearn.metrics.silhouette_score(blobs, km.labels_)) # Esta la dispersión de nuestros grupos

In [ ]:
plt.plot(
    n_clusters_iterable,
    silhouette_scores
)

Según los análisis anteriores podemos desprender que alrededor de 5 grupos sería un hiperparámetro óptimo para nuestro modelamiento.

In [ ]:
km_tuned = sklearn.cluster.KMeans(n_clusters=5, random_state=11) # Instanciamos el algoritmo de k-means
km_tuned.fit(blobs) # Ajustamos el modelo de k-means

In [ ]:
km_tuned.labels_ # Grupos a los cuales asoció el modelo a cada una de las instancias del conjunto de datos

In [ ]:
plt.scatter(blobs.x, blobs.y, c = km_tuned.labels_) # Verificamos gráficamente el agrupamiento

# Clustering jerárquico

Para explorar el funcionamiento del clustering jerárquico utilizaremos un conjunto de datos en donde los grupos no tienen una configuración circular.

In [ ]:
moons = pd.read_csv("https://raw.githubusercontent.com/fvillena/biocompu/2022/data/twomoons.csv")

In [ ]:
moons.head()

In [ ]:
plt.scatter(moons.x, moons.y)

Ajustamos un modelo de k-means para demostrar que el comportamiento de este modelo no es el más correcto.

In [ ]:
km_moons = sklearn.cluster.KMeans(n_clusters=2, random_state=11)
km_moons.fit(moons)

In [ ]:
km.labels_

Observamos que el agrupamiento que se realizó con k-means no se ajusta a nuestro conjunto de datos.

In [ ]:
plt.scatter(moons.x, moons.y, c = km_moons.labels_)

Ajustamos un modelo de clustering jerárquico.

In [ ]:
ac = sklearn.cluster.AgglomerativeClustering()
ac.fit(moons)

In [ ]:
plt.scatter(moons.x, moons.y, c = ac.labels_)

Nuestro modelo tampoco se comporta como esperamos porque existe un hiperparámetro que debemos ajustar. El clustering jerárquico puede utilizar distintos métodos para poder medir la distancia entre clusters y así poder unirlos o no. Evaluemos el funcionamiento de nuestro modelo al visualizar el agrupamiento utilizando los distintos métodos de enlace.

In [ ]:
linkage_methods = ["complete", "average", "single"]
fig, axs = plt.subplots(ncols=len(linkage_methods))
for i,l in enumerate(linkage_methods):
    ac_current = sklearn.cluster.AgglomerativeClustering(linkage=l)
    ac_current.fit(moons)
    axs[i].scatter(moons.x, moons.y, c = ac_current.labels_)
    axs[i].set_title(l)
    axs[i].axis('off')
plt.tight_layout()

Para nuestro conjunto de datos el mejor método es single.

In [ ]:
ac_tuned = sklearn.cluster.AgglomerativeClustering(linkage="single")
ac_tuned.fit(moons)

In [ ]:
plt.scatter(moons.x, moons.y, c = ac_tuned.labels_)

# Datos de microarreglo

Tenemos un conjunto de datos de expresión génica de genes dentro de distintas lineas celulares. Aplicaremos clustering jerárquico para evaluar qué genes son más cercanos entre sí, como también para evaluar qué lineas celulares son más cercanas entre sí.

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)
    # Plot the corresponding dendrogram
    scipy.cluster.hierarchy.dendrogram(linkage_matrix, **kwargs)

In [ ]:
microarray = pd.read_csv("https://raw.githubusercontent.com/fvillena/biocompu/2021/data/gene_expressions.tsv", sep="\t", index_col="gene", nrows=30)

In [ ]:
microarray.head()

In [ ]:
microarray.columns

Agrupación por genes

Realizaremos una agrupación a nivel de genes.

In [ ]:
ac_microarray = sklearn.cluster.AgglomerativeClustering(linkage='average',distance_threshold=0, n_clusters=None)
ac_microarray.fit(microarray)

Es probable que genes que se encuentren dentro del mismo cluster puedan tener funciones similares o pertenecer a tejidos similares. Utilizaremos una visualización llamada dendrograma para observar cómo se forman los grupos encontrados.

In [ ]:
#plot dendrograma genes (para este caso demora por la cantidad de hojas finales)
plt.figure(figsize=(15,7))
plt.title('Dendrograma Genes')
plot_dendrogram(ac_microarray, leaf_rotation=90, leaf_font_size=8, labels=microarray.index)
plt.xlabel("Genes")
plt.show()

Agrupación por líneas celulares

También realizaremos un agrupamiento a nivel de lineas celulares para explorar qué lineas celulares están más cercanas entre sí.

In [ ]:
microarray_trans = microarray.transpose()

In [ ]:
ac_microarray_trans = sklearn.cluster.AgglomerativeClustering(linkage='average',distance_threshold=0, n_clusters=None)
ac_microarray_trans.fit(microarray_trans)

In [ ]:
plt.figure(figsize=(15,7))
plt.title('Dendrograma Lineas Celulares')
plot_dendrogram(ac_microarray_trans, leaf_rotation=90, leaf_font_size=8, labels=microarray_trans.index)
plt.xlabel("Lineas Celulares")
plt.show()